In [1]:
from agents.email_agent import EmailAgent

a = EmailAgent()
a({'input':'test'})

/home/tp/smartlead_agent/agents/email_agent.py:51: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(
/home/tp/smartlead_agent/agents/email_agent.py:56: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  return initialize_agent(
/home/tp/smartlead_agent/agents/email_agent.py:18: LangChainDep



> Entering new AgentExecutor chain...
I'm here to assist you. How can I help you today?

> Finished chain.


{'input': 'test',
 'memory': [],
 'output': "I'm here to assist you. How can I help you today?"}

In [ ]:
from langchain.document_loaders import PyMuPDFLoader

# Load a PDF file
loader = PyMuPDFLoader("data/Hyred_Intro-deck.pdf")
documents = loader.load()

# Print extracted text
for doc in documents:
    print(doc.page_content)  # Extracted text

In [26]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from pprint import pprint
# Base URL to crawl
base_url = "https://hyred.team"
visited_links = set()  # To avoid duplicate links

def extract_links(url):
    """Extract all internal links from a given URL."""
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code != 200:
            print(f"Failed to access {url}")
            return []
        
        soup = BeautifulSoup(response.text, "html.parser")
        links = set()

        for a_tag in soup.find_all("a", href=True):
            full_link = urljoin(base_url, a_tag["href"])  # Convert relative to absolute URL
            if full_link.startswith(base_url):  # Ensure it's an internal link
                links.add(full_link)
        
        return links
    except Exception as e:
        print(f"Error extracting links from {url}: {e}")
        return []

def extract_main_content(url):
    """Extract main content while removing menus, sidebars, and footers."""
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code != 200:
            print(f"Failed to access {url}")
            return ""

        soup = BeautifulSoup(response.text, "html.parser")

        # Remove unwanted elements (navigation, footers, sidebars, ads)
        for tag in soup.select("nav, footer, aside, header, .sidebar, .menu, .advertisement, .ads"):
            tag.extract()

        # Extract only main content
        main_content = soup.find("main")  # Most sites wrap content in <main>
        if not main_content:
            main_content = soup.find("article")  # Try <article> if <main> is missing
        if not main_content:
            main_content = soup.find("div", class_="content")  # Some sites use <div class="content">

        # Extract text
        if main_content:
            return main_content.get_text(separator=" ", strip=True)
        else:
            return soup.get_text(separator=" ", strip=True)  # Fallback if no clear main content
    except Exception as e:
        print(f"Error extracting content from {url}: {e}")
        return ""

# Start crawling from the base URL
links_to_crawl = extract_links(base_url)
s = []
for link in links_to_crawl:
    if link not in visited_links:
        visited_links.add(link)
        print(f"\nCrawling: {link}")
        page_text = extract_main_content(link)
        s.append(page_text)  # Print only the first 500 characters


Crawling: https://hyred.team#content

Crawling: https://hyred.team/

Crawling: https://hyred.team

Crawling: https://hyred.team/employers-eor-revamp/

Crawling: https://hyred.team#inquiryform

Crawling: https://hyred.team#contactus

Crawling: https://hyred.team/aboutus-revamp/

Crawling: https://hyred.team/employers-recruitment-revamp/

Crawling: https://hyred.team/employers-outsourced-payroll-revamp/


In [54]:
links_to_crawl

{'https://hyred.team',
 'https://hyred.team#contactus',
 'https://hyred.team#content',
 'https://hyred.team#inquiryform',
 'https://hyred.team/',
 'https://hyred.team/aboutus-revamp/',
 'https://hyred.team/employers-eor-revamp/',
 'https://hyred.team/employers-outsourced-payroll-revamp/',
 'https://hyred.team/employers-recruitment-revamp/'}

In [55]:
# Join all strings in the list with newlines and write to file
list_links = list(visited_links)
with open('crawled_content.txt', 'w', encoding='utf-8') as f:
    for i in range(len(s)):
        f.write(list_links[i] + '\n')
        text_split = s[i].split('. ')
        for j in range(len(text_split)):
            f.write(text_split[j] + '\n')
    f.close()
    

In [35]:
s[0].split('. ')

['Where Talent meets Opportunity and Jobs become Careers',
 'We provide an efficient and connected experience for our clients and candidates, seeking to listen to and fully understand their aspirations',
 'The Employment Agency for everyone',
 "Find Opportunities Find Top Talent Recruitment Find the right person for the right job with Hyred's tailored recruitment process",
 'Flexible and efficient',
 "More Info Employer of Record Hire great talent anywhere, anytime with Hyred's Employer of Record solution",
 'Compliant and hassle free',
 "More Info Outsorced payroll Never worry about payroll so that you can focus on what's most important - growing your business",
 "More Info Let's break it down Leading HR Solutions Firm in South East Asia SEE MORE Trusted by global brands across the region SEE MORE Comperhensive talent management solutions - global yet local SEE MORE End-to-end employment lifecycle - Recruitment to Employer of Record to Payroll SEE MORE Testimonial Ravindran Khrisnan |